In [43]:
import pandas as pd 
import pathlib
import os
from docx import Document
from docx.text.parfmt import ParagraphFormat 
from docx.shared import Inches, RGBColor

NUMBER_OF_PEOPLE = 5

In [73]:
### Source: https://github.com/python-openxml/python-docx/issues/74#issuecomment-261169410

import docx
def add_hyperlink(paragraph, url, text, color, underline):
    """
    A function that places a hyperlink within a paragraph object.

    :param paragraph: The paragraph we are adding the hyperlink to.
    :param url: A string containing the required url
    :param text: The text displayed for the url
    :return: The hyperlink object
    """

    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element
    new_run = docx.oxml.shared.OxmlElement('w:r')

    # Create a new w:rPr element
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Add color if it is given
    if not color is None:
      c = docx.oxml.shared.OxmlElement('w:color')
      c.set(docx.oxml.shared.qn('w:val'), color)
      rPr.append(c)

    # Remove underlining if it is requested
    if not underline:
      u = docx.oxml.shared.OxmlElement('w:u')
      u.set(docx.oxml.shared.qn('w:val'), 'none')
      rPr.append(u)

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink

In [35]:
path = r"C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\Data Set\Filtered and SQL\Filtered MDF.csv"
df = pd.read_csv(path)

df = df.sample(frac=1).reset_index(drop=True)
df["Number_of_Paragraphs"]

0       6.953333
1       7.766667
2       8.673333
3      21.960000
4       6.380000
         ...    
395     6.460000
396     7.093333
397    48.326667
398    13.300000
399     5.966667
Name: Number_of_Paragraphs, Length: 400, dtype: float64

In [68]:
"""
Note: Remember to rerun this before every rerun of the cell below. This because the document object needs to be renewed for every time this is done. 
I will definitely make this a function once I finalize everything else. 
"""

#create document files for each subreddit in 
docs = []
    
[docs.append(Document()) for i in range(NUMBER_OF_PEOPLE)] 

#create dataframe for each person on the team
dfs = []

#randomize indexes to adjust for the length of =

index_num = [x for x in range(0, df.shape[0]+1,df.shape[0]//NUMBER_OF_PEOPLE)]

for i in range(0, len(index_num)-1, 1): #could be improved
    dfs.append(df.iloc[index_num[i]:index_num[i+1]])


In [72]:
count = 0 
for i in range(len(dfs)): 

    for index in range(len(dfs[i])): 
        # first line of each narrative which contains the link to the submission and the subreddit
        firstLine = docs[count].add_paragraph("Post Url: ")
        add_hyperlink(firstLine, dfs[i].iloc[index]['url'], 'Link to Post', '00a7ff', False) ##doesn't work at the moment
        firstLine.add_run(', Subreddit: r/'+ dfs[i].iloc[index]['subreddit'])

        #paragraph and format
        para = docs[count].add_paragraph(dfs[i].iloc[index]['selftext'])
        para.paragraph_format.first_line_indent = Inches(0.5)
        para.paragraph_format.right_indent = Inches(0.5)
        para.keep_together = True
        docs[count].add_page_break() 

    docs[count].save("Person_" + str(count)+ ".docx")
    count += 1

In [ ]:
for doc in docs: 
    builder = aw.DocumentBuilder(doc)
    count = 0
    font = builder.font
    font.size = 12
    for frame in dfs:
        for index in range(len(frame)):  
            builder.insert_hyperlink("Link", df.loc[i,'url'], False)
            builder.write(" ,Subreddit: " + df.loc[i, 'subreddit'])
            builder.writeln("")

            #setting paragraph formatting
            paragraphFormat = builder.paragraph_format
            paragraphFormat.left_indent = 8
            paragraphFormat.first_line_indent = 8
            paragraphFormat.alignment = aw.ParagraphAlignment.JUSTIFY
            paragraphFormat.keep_together = True
            builder.writeln(df.loc[i, 'selftext'])


    doc.save("Person_" + str(count)+ ".docx")
    count += count
